# Functions

<hr style="clear:both">
This notebook creates all functions that help us during the simulation. This way, the main code stays readable.

**Authors:** [Lynn Fayed](https://people.epfl.ch/lynn.fayed), [Lorenzo Ballinari](https://people.epfl.ch/lorenzo.ballinari), [Paulo Alexandre Ribeiro de Carvalho](https://people.epfl.ch/paulo.ribeirodecarvalho)

<hr style="clear:both">

## Librairies

In [1]:
import numpy as np

## Creation of all needed functions

In [2]:
def compute_speed(accumulation):
    """ Compute network speed based on accumulation """

    m = accumulation / 1000
    tveh = 60
    if m <= 0.6 * tveh:
        velocity = 30.8 * np.exp(-m * 0.145 * 20 / tveh)
    elif m <= tveh:
        velocity = 5.4 - (m - 0.6 * tveh) * 0.71 * 20 / tveh
    else:
        velocity = 0
    if velocity < 0:
        velocity = 0
    velocity = 36 / 30.8 * velocity
    return velocity

In [3]:
def compute_request_arrivals(waiting_requests, sharing_fraction, request_arrivals):
    """ Generate rider arrivals """

    number_of_arrivals = len(request_arrivals[0])
    waiting_requests.request_ID = np.append(waiting_requests.request_ID, request_arrivals[4, :].astype(int))
    waiting_requests.origin = np.append(waiting_requests.origin, request_arrivals[1, :].astype(int))
    waiting_requests.destination = np.append(waiting_requests.destination, request_arrivals[2, :].astype(int))
    waiting_requests.trip_length = np.append(waiting_requests.trip_length, request_arrivals[3, :])
    waiting_requests.request_arrival_time = np.append(waiting_requests.request_arrival_time, request_arrivals[0, :])
    r = np.random.rand(number_of_arrivals)
    r = r < sharing_fraction
    nr = r == False
    waiting_requests.willingness_to_share = np.append(waiting_requests.willingness_to_share, r.astype(int))
    waiting_requests.assigned_driver = np.append(waiting_requests.assigned_driver, [-1] * number_of_arrivals)
    waiting_requests.traveled_time = np.append(waiting_requests.traveled_time, [0] * number_of_arrivals)

In [4]:
def compute_private_vehicle_arrivals(private_vehicles, private_vehicle_arrivals):
    """ Generate private vehicle arrivals """

    number_of_arrivals = len(private_vehicle_arrivals[0])
    private_vehicles.vehicle_ID = np.append(private_vehicles.vehicle_ID, private_vehicle_arrivals[4, :].astype(int))
    private_vehicles.remaining_distance = np.append(private_vehicles.remaining_distance, private_vehicle_arrivals[3, :])
    private_vehicles.vehicle_arrival_time = np.append(private_vehicles.vehicle_arrival_time, private_vehicle_arrivals[0, :])

In [5]:
def update_ride_hailing_vehicle(ride_hailing_vehicles, vehicle_ID):
    """ Update the tncs status at every pick up/drop off """

    attributes = ['destination_IDs', 'destination_actions', 'current_destinations']
    activity = ride_hailing_vehicles.destination_actions[0, vehicle_ID]
    ride_hailing_vehicles.occupancy[vehicle_ID] = ride_hailing_vehicles.occupancy[vehicle_ID] + 1 if activity == 1 else ride_hailing_vehicles.occupancy[vehicle_ID] - 1
    for attribute in attributes:
        exec(f"ride_hailing_vehicles.{attribute}[:, vehicle_ID] = np.roll(ride_hailing_vehicles.{attribute}[:, vehicle_ID], -1)")
        exec(f"ride_hailing_vehicles.{attribute}[-1, vehicle_ID] = 0")

In [6]:
def pick_up(traveling_passengers, waiting_requests, passenger_index):
    """ Update the waiting requests and traveling passengers after a pick up """

    # Adding the passenger in the traveling class
    for attribute in traveling_passengers.__dict__:
        exec(f"traveling_passengers.{attribute} = np.append(traveling_passengers.{attribute}, waiting_requests.{attribute}[passenger_index])")

    # Taking the passenger out of the waiting class:
    for attribute in waiting_requests.__dict__:
        exec(f"waiting_requests.{attribute} = np.delete(waiting_requests.{attribute}, passenger_index)")
        
    # update l'attribut du temps déjà passé par le voyageur

In [7]:
def drop_off(traveling_passengers, passenger_index):
    """ Update traveling passengers after drop off """

    for attribute in traveling_passengers.__dict__:
        # Taking traveling passenger out of the system:
        exec(f"traveling_passengers.{attribute} = np.delete(traveling_passengers.{attribute}, passenger_index)")

In [8]:
def abandon_requests(waiting_requests, private_vehicles, exiting_bool):
    """ Remove abandoning vehicles from list of passengers and add them to pvs """
    
    private_vehicles.vehicle_ID = np.append(private_vehicles.vehicle_ID, waiting_requests.request_ID[exiting_bool])
    private_vehicles.remaining_distance = np.append(private_vehicles.remaining_distance, waiting_requests.trip_length[exiting_bool])
    private_vehicles.vehicle_arrival_time = np.append(private_vehicles.vehicle_arrival_time, waiting_requests.request_arrival_time[exiting_bool])

    remaining_requests = ~exiting_bool
    for attribute in waiting_requests.__dict__:
        exec(f"waiting_requests.{attribute} = waiting_requests.{attribute}[remaining_requests]")

In [9]:
def complete_private_vehicle_trips(private_vehicles):
    """ Remove private vehicles that complete their trips """

    remaining_private_vehicle_indices = private_vehicles.remaining_distance > 0
    for attribute in private_vehicles.__dict__:
        exec(f"private_vehicles.{attribute} = private_vehicles.{attribute}[remaining_private_vehicle_indices]")

In [10]:
def update_remaining_distance(ride_hailing_vehicles, vehicle_index, traveled_distance):
    """ Update remaining distance after every time step """
    
    ride_hailing_vehicles.remaining_distance[vehicle_index] -= traveled_distance


In [11]:
def match(waiting_requests, ride_hailing_vehicles, speed, fleet_size, time, maximum_waiting_time):
    """ Matching passengers using bipartite matching """
    
    cost_matrix = np.ones((fleet_size, len(waiting_requests.request_ID))) * 10000 #initialise la matrice des cost à minimizer par la suite.
    unassigned_request_indices = (waiting_requests.assigned_driver == -1).nonzero()[0] #crée un vecteur qui a les indice de toutes les requests non assigné à une voiture. 

    if len(unassigned_request_indices) == 0: #check si il y a des request à traiter, si non on s'arrête.
        return

    for vehicle_ID in range(fleet_size): #on prend chaque voiture de la flotte une à une
        if (ride_hailing_vehicles.occupancy[vehicle_ID] == 0) and (ride_hailing_vehicles.destination_actions[0, vehicle_ID] == 0):
            for unassigned_request_index in unassigned_request_indices:
                pick_up_distance = alldists[ride_hailing_vehicles.next_node[vehicle_ID], waiting_requests.origin[unassigned_request_index]] + ride_hailing_vehicles.distance_to_next_node[vehicle_ID]
                if time + pick_up_distance / speed < waiting_requests.request_arrival_time[unassigned_request_index] + maximum_waiting_time:
                    cost_matrix[vehicle_ID, unassigned_request_index] = pick_up_distance

    assigned_vehicles, assigned_requests = linear_sum_assignment(cost_matrix) #fonction permettant d'associer les solutions optimales (donc lié une voiture(index de la ligne) à une request(index de la colonne))
    if len(assigned_vehicles) == 0: #check si il y a eu des solutions de matching trouvées, si non on s'arrête.
        return

    for assignment in range(len(assigned_vehicles)):
        assigned_vehicle = assigned_vehicles[assignment]
        assigned_request = assigned_requests[assignment]
        if cost_matrix[assigned_vehicle, assigned_request] != 10000:
            ride_hailing_vehicles.destination_actions[:2, assigned_vehicle] = [1, 2]
            ride_hailing_vehicles.destination_IDs[:2, assigned_vehicle] = [waiting_requests.request_ID[assigned_request]] * 2
            ride_hailing_vehicles.current_destinations[:2, assigned_vehicle] = [waiting_requests.origin[assigned_request], waiting_requests.destination[assigned_request]]
            ride_hailing_vehicles.remaining_distance[assigned_vehicle] = alldists[ride_hailing_vehicles.next_node[assigned_vehicle], ride_hailing_vehicles.current_destinations[0, assigned_vehicle]]
            waiting_requests.assigned_driver[assigned_request] = assigned_vehicle

In [12]:
def update_network(ride_hailing_vehicles, waiting_requests, traveling_passengers, vehicle_ID):
    """ Update vehicle positions and perform pick up and drop off operations """

    if (ride_hailing_vehicles.remaining_distance[vehicle_ID] <= 0 and ride_hailing_vehicles.destination_actions[0, vehicle_ID] == 0):
        ride_hailing_vehicles.remaining_distance[vehicle_ID] = 0
        ride_hailing_vehicles.distance_to_next_node[vehicle_ID] = 0
        return
    elif ride_hailing_vehicles.remaining_distance[vehicle_ID] > 0:
        move_vehicle(ride_hailing_vehicles, vehicle_ID)
        return
    else:
        if ride_hailing_vehicles.destination_actions[1, vehicle_ID] == 0:
            ride_hailing_vehicles.remaining_distance[vehicle_ID] = 0
            ride_hailing_vehicles.distance_to_next_node[vehicle_ID] = 0
            ride_hailing_vehicles.next_node[vehicle_ID] = ride_hailing_vehicles.current_destinations[0, vehicle_ID]
        else:
            ride_hailing_vehicles.remaining_distance[vehicle_ID] += alldists[ride_hailing_vehicles.current_destinations[0, vehicle_ID], ride_hailing_vehicles.current_destinations[1, vehicle_ID]]
        passenger_index = np.where((waiting_requests.request_ID == ride_hailing_vehicles.destination_IDs[0, vehicle_ID])) if ride_hailing_vehicles.destination_actions[0, vehicle_ID] == 1 \
            else np.where((traveling_passengers.request_ID == ride_hailing_vehicles.destination_IDs[0, vehicle_ID]))
        pick_up(traveling_passengers, waiting_requests, passenger_index) if ride_hailing_vehicles.destination_actions[0, vehicle_ID] == 1 else drop_off(traveling_passengers, passenger_index)
        ride_hailing_vehicles.next_node[vehicle_ID] = ride_hailing_vehicles.current_destinations[0, vehicle_ID] # next node is the current one
        update_ride_hailing_vehicle(ride_hailing_vehicles, vehicle_ID)  # this should always come after updating wreq        
        update_network(ride_hailing_vehicles, waiting_requests, traveling_passengers, vehicle_ID)

In [13]:
def move_vehicle(ride_hailing_vehicles, vehicle_ID):
    """ Move vehicle on their path before reaching their final destinations """

    if ride_hailing_vehicles.remaining_distance[vehicle_ID] >= alldists[ride_hailing_vehicles.next_node[vehicle_ID], ride_hailing_vehicles.current_destinations[0, vehicle_ID]]:
        ride_hailing_vehicles.distance_to_next_node[vehicle_ID] = ride_hailing_vehicles.remaining_distance[vehicle_ID] - alldists[ride_hailing_vehicles.next_node[vehicle_ID], ride_hailing_vehicles.current_destinations[0, vehicle_ID]]
        return
    
    else:
        while ride_hailing_vehicles.remaining_distance[vehicle_ID] < alldists[ride_hailing_vehicles.next_node[vehicle_ID], ride_hailing_vehicles.current_destinations[0, vehicle_ID]]:
            ride_hailing_vehicles.next_node[vehicle_ID] = allpaths[ride_hailing_vehicles.current_destinations[0, vehicle_ID], ride_hailing_vehicles.next_node[vehicle_ID]]
            ride_hailing_vehicles.distance_to_next_node[vehicle_ID] = ride_hailing_vehicles.remaining_distance[vehicle_ID] - alldists[ride_hailing_vehicles.next_node[vehicle_ID], ride_hailing_vehicles.current_destinations[0, vehicle_ID]]
        return

In [14]:
def occupancy(ride_hailing_vehicles):
    """Check the occupancy of every vehicle and give back only the available ones"""
    
    # Initialize the list that will contain the vehicles available for sharing.
    index_vehicles_available_for_sharing : list = []
        
    for i in ride_hailing_vehicles.vehicle_ID:
        
        # we treat the first scenario here.
        if ride_hailing_vehicles.occupancy[i]==0 and ride_hailing_vehicles.destination_actions[0,i]==1: 
            # we check that there is no second request already assigned to this driver.
            if ride_hailing_vehicles.destination_actions[2,i]==0:
                index_vehicles_available_for_sharing.append(i)
            
        # we treat the second scenario here.
        elif ride_hailing_vehicles.occupancy[i]==1 and ride_hailing_vehicles.destination_actions[0,i]==2: 
            # we check that there is no second request already assigned to this driver.
            if ride_hailing_vehicles.destination_actions[1,i]==0:
                index_vehicles_available_for_sharing.append(i)
                
    return index_vehicles_available_for_sharing

In [15]:
def check_waiting_time(request_ID, vehicle_ID, scenario, ride_hailing_vehicles, waiting_requests, 
                       traveling_passengers, input_variables, time):
    """Check if the waiting time constraint is feasible or not for each scenario"""
    
    check=0
    # conditions of the scenarios for SCENARIO 1
    if ride_hailing_vehicles.destination_IDs[0,vehicle_ID] in waiting_requests.request_ID:
        if ride_hailing_vehicles.occupancy[vehicle_ID]==0:
            if ride_hailing_vehicles.destination_actions[0,vehicle_ID]==1:
                waiting_ID = np.where(waiting_requests.request_ID == ride_hailing_vehicles.destination_IDs[0,vehicle_ID])
                waiting_ID = int(waiting_ID[0])
                                
                if waiting_requests.willingness_to_share[waiting_ID]==1: # We check that the passenger already assigned to a driver wants to share too.
                    current_waiting_time_1 = time - waiting_requests.request_arrival_time[request_ID]
                    current_waiting_time_2 = time - waiting_requests.request_arrival_time[waiting_ID]

                    
# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––                    
# –––––––––––––––––––––––––––––––––––––––––––– scenario 1&2 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––                    
# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
                    
                    # distance/waiting time for scenario 1&2 (pick-up 1 and 2)
                    if scenario == 1 or scenario == 2:
                        distance_1 = alldists[ride_hailing_vehicles.next_node[vehicle_ID],waiting_requests.origin[request_ID]] + ride_hailing_vehicles.distance_to_next_node[vehicle_ID]
                        waiting_time_1 = current_waiting_time_1 + distance_1/velocity[-1]
                        distance_2 = alldists[waiting_requests.origin[request_ID],ride_hailing_vehicles.current_destinations[0,vehicle_ID]]
                        waiting_time_2 = current_waiting_time_2 + distance_1/velocity[-1] + distance_2/velocity[-1]
                        
                        if waiting_time_1 < input_variables.maximum_waiting_time and waiting_time_2 < input_variables.maximum_waiting_time:
                            check = 1
                        
                            return check, waiting_time_1, waiting_time_2


# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––                    
# –––––––––––––––––––––––––––––––––––––––––––– scenario 3&4 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––                    
# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
                                     
                    # distance/waiting time for scenario 4&5 (pick-up 2 and 1)
                    if scenario == 3 or scenario == 4:    
                        distance_2 = ride_hailing_vehicles.remaining_distance[vehicle_ID]
                        waiting_time_2 = current_waiting_time_2 + distance_2/velocity[-1]
                        distance_1 = alldists[ride_hailing_vehicles.current_destinations[0,vehicle_ID],waiting_requests.origin[request_ID]]
                        waiting_time_1 = current_waiting_time_1 + distance_2/velocity[-1] + distance_1/velocity[-1]

                        if waiting_time_1 < input_variables.maximum_waiting_time and waiting_time_2 < input_variables.maximum_waiting_time:
                            check = 1

                            return check, waiting_time_1, waiting_time_2                   



    # conditions of the scenarios for SCENARIO 2
    if ride_hailing_vehicles.destination_IDs[0,vehicle_ID] in traveling_passengers.request_ID:
        if ride_hailing_vehicles.occupancy[vehicle_ID]==1 :
            if ride_hailing_vehicles.destination_actions[0,vehicle_ID]==2:
                traveler_ID = np.where(traveling_passengers.request_ID==ride_hailing_vehicles.destination_IDs[0,vehicle_ID])
                traveler_ID = int(traveler_ID[0])
                
                if traveling_passengers.willingness_to_share[traveler_ID]==1: # We check that the passenger already assigned to a driver wants to share too.
                    current_waiting_time_1 = time - waiting_requests.request_arrival_time[request_ID]

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––                    
# –––––––––––––––––––––––––––––––––––––––––––– scenario 5&6 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––                    
# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––                                       
                    
                    # distance/waiting time for scenario 5&6 (only pick-up 1)
                    if scenario == 5 or scenario == 6:
                        distance = alldists[ride_hailing_vehicles.next_node[vehicle_ID],waiting_requests.origin[request_ID]] + ride_hailing_vehicles.distance_to_next_node[vehicle_ID]
                        waiting_time = current_waiting_time_1 + distance/velocity[-1]

                        if waiting_time < input_variables.maximum_waiting_time :
                            check = 1
                            
                            return check, waiting_time, 0

                    
    return check, 1000, 1000

In [16]:
def check_detour_time(request_ID, vehicle_ID, scenario, ride_hailing_vehicles, waiting_requests, traveling_passengers, 
                      input_variables, time, velocity, check_w):
    """Check if the waiting time constraint is feasible or not for each scenario"""
    
    check=0
    
    # if the condition for waiting_time is not feasible, it is not useful to enter in this function, so we stop.
    if check_w == 1:
        
        # conditions of the scenarios for SCENARIO 1
        if ride_hailing_vehicles.destination_IDs[0,vehicle_ID] in waiting_requests.request_ID:
            if ride_hailing_vehicles.occupancy[vehicle_ID]==0:
                if ride_hailing_vehicles.destination_actions[0,vehicle_ID]==1:
                    
                    #np.where() return the index where a value is found in a array
                    waiting_ID = np.where(waiting_requests.request_ID == ride_hailing_vehicles.destination_IDs[0,vehicle_ID])
                    waiting_ID = int(waiting_ID[0])

                    # We check that the passenger already assigned to a driver wants to share too.
                    if waiting_requests.willingness_to_share[waiting_ID]==1:

                        # Determine all origin and destination
                        p1 = waiting_requests.origin[request_ID]
                        p2 = waiting_requests.origin[waiting_ID]
                        d1 = waiting_requests.destination[request_ID]
                        d2 = waiting_requests.destination[waiting_ID]

                        # Compute all the distances
                        p1_p2 = alldists[p1,p2]
                        p2_d1 = alldists[p2,d1]
                        d1_d2 = alldists[d1,d2]
                        p2_d2 = alldists[p2,d2]
                        p1_d1 = alldists[p1,d1]
                        p1_d2 = alldists[p1,d2]

    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––– scenario 1 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

                        if scenario == 1:
                            trip_length_1 = p1_p2 + p2_d1
                            trip_length_2 = p2_d1 + d1_d2
                    
                            detour_time_1 = abs((trip_length_1-p1_d1)/velocity)
                            detour_time_2 = abs((trip_length_2-p2_d2)/velocity)

                            # check the detour time constraint
                            if detour_time_1 < input_variables.maximum_detour_time and detour_time_2 < input_variables.maximum_detour_time: 
                                check=1

                                return check, detour_time_1, detour_time_2

    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––– scenario 2 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

                        # this scenario only has a detour for passenger 2
                        if scenario == 2:
                            trip_length_1 = p1_p2 + p2_d2 + d1_d2
                            trip_length_2 = p2_d2

                            detour_time_1 = abs((trip_length_1-p1_d1)/velocity)
                            detour_time_2 = abs((trip_length_2-p2_d2)/velocity)

                            # check the detour time constraint
                            if detour_time_1 < input_variables.maximum_detour_time and detour_time_2 < input_variables.maximum_detour_time: 
                                check=1

                                return check, detour_time_1, detour_time_2


    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––– scenario 3 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

                        # this scenario has no detour for passenger 1
                        if scenario == 3:
                            trip_length_1 = p1_d1
                            trip_length_2 = p1_p2 + p1_d1 + d1_d2

                            detour_time_1 = abs((trip_length_1-p1_d1)/velocity)
                            detour_time_2 = abs((trip_length_2-p2_d2)/velocity)

                            # check the detour time constraint
                            if detour_time_1 < input_variables.maximum_detour_time and detour_time_2 < input_variables.maximum_detour_time: 
                                check=1

                                return check, detour_time_1, detour_time_2

    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––– scenario 4 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

                        # this scenario has no detour for passenger 1
                        if scenario == 4:
                            trip_length_1 = p1_d2 + d1_d2
                            trip_length_2 = p1_p2 + p1_d2

                            detour_time_1 = abs((trip_length_1-p1_d1)/velocity)
                            detour_time_2 = abs((trip_length_2-p2_d2)/velocity)

                            # check the detour time constraint
                            if detour_time_1 < input_variables.maximum_detour_time and detour_time_2 < input_variables.maximum_detour_time: 
                                check=1

                                return check, detour_time_1, detour_time_2


        # conditions of the scenarios for SCENARIO 2
        if ride_hailing_vehicles.destination_IDs[0,vehicle_ID] in traveling_passengers.request_ID:
            if ride_hailing_vehicles.occupancy[vehicle_ID]==1 :
                if ride_hailing_vehicles.destination_actions[0,vehicle_ID]==2:
                    traveler_ID = np.where(traveling_passengers.request_ID == ride_hailing_vehicles.destination_IDs[0,vehicle_ID])
                    traveler_ID = int(traveler_ID[0])

                    if traveling_passengers.willingness_to_share[traveler_ID]==1: # We check that the passenger already assigned to a driver wants to share too.

                        # Determine all origin and destination
                        p1 = waiting_requests.origin[request_ID]
                        p2 = traveling_passengers.origin[traveler_ID]
                        c2 = ride_hailing_vehicles.next_node[vehicle_ID] # position of the car
                        d1 = waiting_requests.destination[request_ID]
                        d2 = traveling_passengers.destination[traveler_ID]

                        # Compute all the distances 
                        d1_d2 = alldists[d1,d2]
                        p1_d1 = alldists[p1,d1]
                        p1_d2 = alldists[p1,d2]
                        p1_c2 = alldists[p1,c2] + ride_hailing_vehicles.distance_to_next_node[vehicle_ID]
                        c2_d2 = alldists[c2,d2] + ride_hailing_vehicles.distance_to_next_node[vehicle_ID]
                        p2_d2 = alldists[p2,d2]

    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––– scenario 5 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

                        if scenario == 5:
                            trip_length_1 = p1_d2 + d1_d2
                            trip_length_2 = p1_c2 + p1_d2

                            detour_time_1 = abs((trip_length_1-p1_d1)/velocity)
                            detour_time_2 = abs((trip_length_2/velocity) + traveling_passengers.traveled_time[traveler_ID] - (p2_d2/velocity))

                            # check the detour time constraint
                            if detour_time_1 < input_variables.maximum_detour_time and detour_time_2 < input_variables.maximum_detour_time: 
                                check=1

                                return check, detour_time_1, detour_time_2

    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––– scenario 6 –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
    # –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

                        if scenario == 6:
                            trip_length_1 = p1_d1
                            trip_length_2 = p1_c2 + p1_d1 + d1_d2

                            detour_time_1 = abs((trip_length_1-p1_d1)/velocity)
                            detour_time_2 = abs((trip_length_2/velocity) + traveling_passengers.traveled_time[traveler_ID] - (p2_d2/velocity))

                            # check the detour time constraint
                            if detour_time_1 < input_variables.maximum_detour_time and detour_time_2 < input_variables.maximum_detour_time: 
                                check=1

                                return check, detour_time_1, detour_time_2
                    
                    
    return check, 1000, 1000

In [17]:
def update_matrix(request_ID, vehicle_ID, cost_matrix, scenario_matching, metrics, metrics_feasibilty, 
                  index_vehicles_available_for_sharing, index_unassigned_request_ready_to_share):
    """ Update the cost matrix and stock the scenario that give this cost """
    
    # Index to update the matrix
    matrix_index_veh = index_vehicles_available_for_sharing.index(vehicle_ID) # this variable is a list, so we use list.index(...)
    
    matrix_index_request = np.where(index_unassigned_request_ready_to_share == request_ID) # this variable is a np.array, so we use np.where(...)
    matrix_index_request = int(matrix_index_request[0]) # We convert the value to a int
    
    # Choose the smallest value (feasible) and its scenario
    smallest_metric = 1000
    scenario = 0
    for i in range(len(metrics)):
        if metrics_feasibilty[i]==1 and metrics[i]<smallest_metric:
            smallest_metric = metrics[i]
            scenario = i+1
    
    # Update the matrix
    cost_matrix[matrix_index_veh, matrix_index_request] = smallest_metric
    scenario_matching[matrix_index_veh, matrix_index_request] = scenario
    
    return cost_matrix, scenario_matching

In [18]:
def update_matrix_for_max_ass(request_ID, vehicle_ID, cost_matrix, scenario_matching, metrics, metrics_feasibilty, 
                                     index_vehicles_available_for_sharing, index_unassigned_request_ready_to_share):
    """ Update the cost matrix and stock the scenario that give this cost """
    
    # Index to update the matrix
    matrix_index_veh = index_vehicles_available_for_sharing.index(vehicle_ID) # this variable is a list, so we use list.index(...)
    
    matrix_index_request = np.where(index_unassigned_request_ready_to_share == request_ID) # this variable is a np.array, so we use np.where(...)
    matrix_index_request = int(matrix_index_request[0]) # We convert the value to a int
    
    # Choose the smallest value (feasible) and its scenario
    smallest_metric = 1000 # allow us to choose the better scenario
    smallest_metric_update = 1000 # will update the matrix with 1 (where sharing is possible) or 1000 (otherwise)
    scenario = 0
    for i in range(len(metrics)):
        if metrics_feasibilty[i]==1 and metrics[i]<smallest_metric:
            smallest_metric = metrics[i]
            smallest_metric_update = 1
            scenario = i+1
    
    # Update the matrix
    cost_matrix[matrix_index_veh, matrix_index_request] = smallest_metric_update
    scenario_matching[matrix_index_veh, matrix_index_request] = scenario
    
    return cost_matrix, scenario_matching

In [19]:
def match_sharing(waiting_requests, ride_hailing_vehicles, cost_matrix, scenario_matching, Detour_time_perf, traveling_passengers, 
                  input_variables, time, velocity, check_w, history_sharing):
    """ Matching passengers using bipartite matching """
    
    # Function doing the matching (pair of line and columns with the lowest cost)
    assigned_vehicles, assigned_requests = linear_sum_assignment(cost_matrix)
            
    # Check if there is solutions of matching found, if not we stop.
    if len(assigned_vehicles) == 0:
        return history_sharing
    
    else:
        dest_action = []
        dest_IDs = []
        
        for assignment in range(len(assigned_vehicles)):
            index_matrix_vehicle = assigned_vehicles[assignment]
            index_matrix_request = assigned_requests[assignment]
                        
            if cost_matrix[index_matrix_vehicle, index_matrix_request] != 1000:

                # Slicing for the update
                assigned_vehicle = int(index_vehicles_available_for_sharing[index_matrix_vehicle])
                assigned_request = int(index_unassigned_request_ready_to_share[index_matrix_request])
                scenario_index = int(scenario_matching[index_matrix_vehicle, index_matrix_request] - 1)
                scenario = scenario_index + 1
                
                # To change the right number of "case" in our instances (depends of the scenario)
                limit_slicing = 4
                if scenario > 4:
                    limit_slicing = 3

                # ID of both request that are going to share a vehicle.
                ID_1 = waiting_requests.request_ID[assigned_request]
                ID_2 = ride_hailing_vehicles.destination_IDs[0,assigned_vehicle]

                # Origin (Oi) and destination (Di) of both request that are goig to share a vehicle.
                O1 = waiting_requests.origin[assigned_request]
                D1 = waiting_requests.destination[assigned_request]
                # (NB: for scenario of part 2, O2 is the destination : D2 -> O2)
                O2 = ride_hailing_vehicles.current_destinations[0, assigned_vehicle] 
                D2 = ride_hailing_vehicles.current_destinations[1, assigned_vehicle]

                # Array with the right update for each scenario in the destination_actions instance.
                dest_actions = [[1, 1, 2, 2],[1, 1, 2, 2],[1, 1, 2, 2],
                                [1, 1, 2, 2],[1, 2, 2],[1, 2, 2]]

                # Array with the right update for each scenario in the destination_IDs instance.            
                dest_IDs = [[ID_1, ID_2, ID_1, ID_2], [ID_1, ID_2, ID_2, ID_1],[ID_2, ID_1, ID_1, ID_2], 
                            [ID_2, ID_1, ID_2, ID_1],[ID_1, ID_2, ID_1], [ID_1, ID_1, ID_2]]

                # Array with the right update for each scenario in the current_destinations instance.
                curr_dest = [[O1, O2, D1, D2], [O1, O2, D2, D1],[O2, O1, D1, D2], 
                             [O2, O1, D2, D1],[O1, O2, D1], [O1, D1, O2]]
                
                
                # Update the history of sharing scenarios
                add = np.array([[ID_1, ID_2, scenario]])
                history_sharing = np.concatenate([history_sharing, add])
                
                # Update all the necessary instances
                ride_hailing_vehicles.destination_actions[:limit_slicing, assigned_vehicle] = dest_actions[scenario_index]
                ride_hailing_vehicles.destination_IDs[:limit_slicing, assigned_vehicle] = dest_IDs[scenario_index]
                ride_hailing_vehicles.current_destinations[:limit_slicing, assigned_vehicle] = curr_dest[scenario_index]
                ride_hailing_vehicles.remaining_distance[assigned_vehicle] = alldists[ride_hailing_vehicles.next_node[assigned_vehicle], ride_hailing_vehicles.current_destinations[0, assigned_vehicle]]+ride_hailing_vehicles.distance_to_next_node[assigned_vehicle]
                waiting_requests.assigned_driver[assigned_request] = assigned_vehicle
                
        return history_sharing

In [20]:
def occupancy_perf(Occupancy_perf, time, ride_hailing_vehicles, Time):
    """ Keep the evolution of occupancies during all simulation """
    
    time_index = int(np.where(Time == time)[0])
    
    Occupancy_perf[0, time_index] = np.count_nonzero(ride_hailing_vehicles.occupancy == 0)
    Occupancy_perf[1, time_index] = np.count_nonzero(ride_hailing_vehicles.occupancy == 1)
    Occupancy_perf[2, time_index] = np.count_nonzero(ride_hailing_vehicles.occupancy == 2)

In [21]:
def abandonment_perf(Abandonment_perf, abandonment):
    """ Keep the evolution of abondonments during all simulation """
    
    Abandonment_perf = np.append(Abandonment_perf, abandonment)
    
    return Abandonment_perf

In [22]:
def waiting_time_perf(Waiting_time_perf, traveling_passengers, time, already_analyze):
    """ Keep the evolution of the waiting times during all simulation """
    
    for index in range(len(traveling_passengers.request_ID)):
        # We check if the request has not been already analyze.
        if traveling_passengers.request_ID[index] in already_analyze:
            pass
        
        else:
            # We compute the waiting time and stock it and ...
            pick_up_time = time - traveling_passengers.traveled_time[index]
            waiting_time = pick_up_time - traveling_passengers.request_arrival_time[index]
            Waiting_time_perf = np.append(Waiting_time_perf, waiting_time)
            
            # ... we told to the list that this request has been analyze now.
            already_analyze.append(traveling_passengers.request_ID[index])

    return Waiting_time_perf

In [23]:
def detour_time_perf(traveling_passengers, traveling_passengers_old, velocity, requests_are_sharing, Detour_time_perf):
        
    delete_request = []
    
    # Vérifier dans traveling_passengers.assigned_driver si deux requests ont le même driver. Dans quel cas, ils sont ensemble
    # dans cette voiture. Donc on stock leurs ID dans une liste en s'assurant que leur ID ne sont pas déjà dedans. 
    for i in range(len(traveling_passengers.assigned_driver)-1) :
        for j in range(i+1,len(traveling_passengers.assigned_driver)): # le "i" permet ici de ne pas faire plusieurs fois le meme check
            if traveling_passengers.assigned_driver[i]==traveling_passengers.assigned_driver[j]:
                
                if traveling_passengers.request_ID[i] not in requests_are_sharing :
                    requests_are_sharing.append(traveling_passengers.request_ID[i])
                    
                if traveling_passengers.request_ID[j] not in requests_are_sharing :
                    requests_are_sharing.append(traveling_passengers.request_ID[j])
    
    # puis on check si de la liste d'avant, des requests ont quitté cette liste (donc arriver à leur dest)
    for i in range(len(requests_are_sharing)):
        if requests_are_sharing[i] not in traveling_passengers.request_ID : # means the request arrived
            index = np.where(traveling_passengers_old.request_ID == requests_are_sharing[i])
            index = int(index[0])
            
            # Calcul du temps de parcours approximatif si pas de sharing
            origin = traveling_passengers_old.origin[index]
            dest = traveling_passengers_old.destination[index]
            direct_time = alldists[origin,dest]/velocity
            
            # Calcul du trajet avec le sharing
            traveled_time = traveling_passengers_old.traveled_time[index]
            
            # Calcul du détour
            detour_time = abs(traveled_time-direct_time) #abs() because happens to be negative when detour is near 0.
            
            # Update the stock variable
            Detour_time_perf.append(detour_time)
            
            # On stock les requests à supprimer à la fin car elles sont arrivés à destination. 
            delete_request.append(requests_are_sharing[i])
            
    for i in delete_request :     
        requests_are_sharing.remove(i)
        
    # on stock les valeurs de traveling_passengers
    traveling_passengers_old = deepcopy(traveling_passengers)
    
    return traveling_passengers_old

In [24]:
def avg(num_1, num_2):
    average = (num_1 + num_2)/2
    return average

In [25]:
def understand_variables(request_arrivals, ride_hailing_vehicles, waiting_requests, traveling_passengers):
    
    print("request_arrivals\n")
    print("0) request_arrival_time\n",request_arrivals[0,:],request_arrivals[1,:].shape,"\n")
    print("1) origin\n",request_arrivals[1,:],request_arrivals[1,:].shape,"\n")
    print("2) destination\n",request_arrivals[2,:],request_arrivals[1,:].shape,"\n")
    print("3) trip_length\n",request_arrivals[3,:],request_arrivals[1,:].shape,"\n")
    print("4) request_ID\n",request_arrivals[4,:],request_arrivals[1,:].shape,"\n")
    print("\n")

    print("ride_hailing_vehicles\n")
    print("1) vehicle_ID\n",ride_hailing_vehicles.vehicle_ID,ride_hailing_vehicles.vehicle_ID.shape,"\n")
    print("2) next_node\n",ride_hailing_vehicles.next_node,ride_hailing_vehicles.next_node.shape,"\n")
    print("3)remaining_distance\n",ride_hailing_vehicles.remaining_distance,ride_hailing_vehicles.remaining_distance.shape,"\n")
    print("4) destination_IDs\n",ride_hailing_vehicles.destination_IDs,ride_hailing_vehicles.destination_IDs.shape,"\n") #prendre la n-ième colonne -> [:,n]
    print("5) destination_actions\n",ride_hailing_vehicles.destination_actions,ride_hailing_vehicles.destination_actions.shape,"\n")
    print("6) current_destinations\n",ride_hailing_vehicles.current_destinations,ride_hailing_vehicles.current_destinations.shape,"\n")
    print("7) occupancy\n",ride_hailing_vehicles.occupancy,ride_hailing_vehicles.occupancy.shape,"\n")
    print("8) distance_to_next_node\n",ride_hailing_vehicles.distance_to_next_node,ride_hailing_vehicles.distance_to_next_node.shape,"\n")
    print("\n")

    print("waiting_requests\n")
    print("1) request_ID\n",waiting_requests.request_ID,waiting_requests.request_ID.shape,"\n")
    print("2) origin\n",waiting_requests.origin,waiting_requests.origin.shape,"\n")
    print("3) destination\n",waiting_requests.destination,waiting_requests.destination.shape,"\n")
    print("4) willingness_to_share\n",waiting_requests.willingness_to_share,waiting_requests.willingness_to_share.shape,"\n")
    print("5) request_arrival_time\n",waiting_requests.request_arrival_time,waiting_requests.request_arrival_time.shape,"\n")
    print("6) assigned_driver\n",waiting_requests.assigned_driver,waiting_requests.assigned_driver.shape,"\n")
    print("7) trip_length\n",waiting_requests.trip_length,waiting_requests.trip_length.shape,"\n")
    print("8) traveled_time\n",waiting_requests.traveled_time,waiting_requests.traveled_time.shape,"\n")
    print("\n")

    print("traveling_passengers\n")
    print("1) request_ID\n",traveling_passengers.request_ID,traveling_passengers.request_ID.shape,"\n")
    print("2) origin\n",traveling_passengers.origin,traveling_passengers.origin.shape,"\n")
    print("3) destination\n",traveling_passengers.destination,traveling_passengers.destination.shape,"\n")
    print("4) request_arrival_time\n",traveling_passengers.request_arrival_time,traveling_passengers.request_arrival_time.shape,"\n")
    print("5) willingness_to_share\n",traveling_passengers.willingness_to_share,traveling_passengers.willingness_to_share.shape,"\n")
    print("6) assigned_driver\n",traveling_passengers.assigned_driver,traveling_passengers.assigned_driver.shape,"\n")
    print("7) trip_length\n",traveling_passengers.trip_length,traveling_passengers.trip_length.shape,"\n")
    print("8) traveled_time\n",traveling_passengers.traveled_time,traveling_passengers.traveled_time.shape,"\n")